In [1]:
import pygal
import cairosvg
import csv
import codecs
import ast
import csv
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import string
from pygal.style import Style

In [2]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [3]:
def lemmatize_entity(entity):
    result = (entity.lower()).translate(str.maketrans("","", string.punctuation))
    word_list = nltk.word_tokenize(result)
    filtered_output = [w for w in word_list if not w in stop_words]
    lemmatized_key = [lemmatizer.lemmatize(w) for w in filtered_output]
    return lemmatized_key

In [4]:
def read_csv(filename):
    data = []
    with (codecs.open(filename,'r',encoding="utf-8")) as f:
        filereader = csv.reader(f, delimiter=',')
        for row in filereader:
            if(''.join(row).strip()):
                data.append(row)
    f.close()
    data = data[1:]
    return data

In [15]:
data = read_csv('results/all_data_national.csv')

In [7]:
custom_style = Style(
  background = 'white',
  plot_background='white', 
  value_label_font_size = 14,
  value_font_size = 14,
  major_label_font_size = 14 ,
  label_font_size = 14)

In [16]:
def get_graph_input(all_common, cent_crnt_dict, np_crnt_src1, np_crnt_src2,cent_src1,cent_src2,src_name):
    for key in cent_crnt_dict.keys():
        if((match_entity(all_common,key)) is False):
            #print(False,key)
            match_src1 = match_entity(np_crnt_src1,key)
            match_src2 = match_entity(np_crnt_src2,key)
            
            if(match_src1 and match_src2):
                if(src_name=="dd"):
                    all_common[key] = [cent_crnt_dict[key],0,0]
                elif(src_name=="ndtv"):
                    all_common[key] = [0,cent_crnt_dict[key],0]
                elif(src_name=="times"):
                    all_common[key] = [0,0,cent_crnt_dict[key]]
                    
            elif(match_src1 and not match_src2 ):               
                result = match_with_src(cent_src2,key) 
                if((result[0] is not None) and result[1]!=-1):
                    if(src_name=="dd"):
                        all_common[key] = [cent_crnt_dict[key],0,result[1]]
                    if(src_name=="ndtv"):
                        all_common[key] = [0,cent_crnt_dict[key],result[1]]
                    if(src_name=="times"):
                        all_common[key] = [0,result[1],cent_crnt_dict[key]]
                    
                #print("***",result,key)
                #print(match_src1,match_src2,key)
            
            elif(match_src2 and not match_src1):
                result = match_with_src(cent_src2,key) 
                if((result[0] is not None) and result[1]!=-1):
                    if(src_name=="dd"):
                        all_common[key] = [cent_crnt_dict[key],result[1],0]
                    if(src_name=="ndtv"):
                        all_common[key] = [result[1],cent_crnt_dict[key],0]
                    if(src_name=="times"):
                        all_common[key] = [result[1],0,cent_crnt_dict[key]]

    return all_common

In [10]:
def match_with_src(src_dict,entity):
    entity_lemma = lemmatize_entity(entity)
    max_cent = -1
    max_ent = None
    for key in src_dict.keys():
        key_lemma = lemmatize_entity(key)
        if(len(key_lemma)==1 or len(entity_lemma)==1):
            if(len(list(set(key_lemma).intersection(set(entity_lemma))))>0):
                if(src_dict[key]>max_cent):
                    max_cent = src_dict[key]
                    max_ent = key
            #return [key,src_dict[key]]
        else:
            if(len(list(set(key_lemma).intersection(set(entity_lemma))))>1):
                  if(src_dict[key]>max_cent):
                    max_cent = src_dict[key]
                    max_ent = key
            #return [key,src_dict[key]]
    
    return [max_ent,max_cent]

In [11]:
def match_entity(match_dict,entity):
    entity_lemma = lemmatize_entity(entity)
    for key in match_dict.keys():
        key_lemma = lemmatize_entity(key)
        if(len(list(set(key_lemma).intersection(set(entity_lemma))))>0):
            return True
    return False

In [35]:
import pandas as pd
new_df = pd.DataFrame(columns=['ID','dd_all_bubble','ndtv_all_bubble','tn_all_bubble'])

In [36]:
for item in range(0,len(data)):
    if(item%11==0):
        print(item)
    item_id = data[item][0]
    common_cent = ast.literal_eval(data[item][7])
    dd_cent = {x[0]: x[1] for x in ast.literal_eval(data[item][4])}
    ndtv_cent = {x[0]: x[1] for x in ast.literal_eval(data[item][5])}
    times_cent = {x[0]: x[1] for x in ast.literal_eval(data[item][6])}
    dd_tn = {x[0]: x[1] for x in ast.literal_eval(data[item][10])}
    dd_ndtv = {x[0]: x[1] for x in ast.literal_eval(data[item][11])}
    ndtv_dd = {x[0]: x[1] for x in ast.literal_eval(data[item][14])}
    ndtv_tn = {x[0]: x[1] for x in ast.literal_eval(data[item][15])}
    tn_dd = {x[0]: x[1] for x in ast.literal_eval(data[item][12])}
    tn_ndtv = {x[0]: x[1] for x in ast.literal_eval(data[item][13])}
    all_common = {key: common_cent[key] for key in common_cent.keys()}
    all_common = get_graph_input(all_common,dd_cent,dd_tn,dd_ndtv,ndtv_cent,times_cent,"dd")
    all_common = get_graph_input(all_common,ndtv_cent,ndtv_dd,ndtv_tn,dd_cent,times_cent,"ndtv")
    all_common = get_graph_input(all_common,times_cent,tn_dd,tn_ndtv,dd_cent,ndtv_cent,"times")
    #print(all_common)
    
    labels = []
    dd = []
    ndtv = []
    times = []
    
    dd_all_dict = {}
    ndtv_all_dict = {}
    times_all_dict = {}
    
    for key in all_common.keys():
        dd_all_dict[key] = all_common[key][0]
        ndtv_all_dict[key] = all_common[key][1]
        times_all_dict[key] = all_common[key][2]
        labels.append(key)
        dd.append(all_common[key][0])
        ndtv.append(all_common[key][1])
        times.append(all_common[key][2])
    
    #print(dd_all_dict)
    #print(ndtv_all_dict)
    #print(times_all_dict)
    new_df=new_df.append({'ID':item_id,'dd_all_bubble':dd_all_dict,'ndtv_all_bubble':ndtv_all_dict,'tn_all_bubble':times_all_dict},ignore_index=True)
    
    #cht_nm = str(item) + "_" + "common_centrality" + ".png"
    
    """print(labels)
    print(dd)
    print(ndtv)
    print(times)
    
    dot_chart = pygal.Dot(x_label_rotation=90,logarithmic=True,style=custom_style,width=800,height=700)
    dot_chart.title = ''
    dot_chart.x_labels = labels
    dot_chart.add('DD', dd)
    dot_chart.add('NDTV', ndtv)
    dot_chart.add('Times', times)
    dot_chart.render_to_png(cht_nm)"""
    
    

0
11
22
33
44
55
66
77
88
99
110
121
132
143
154
165
176
187
198
209
220
231
242
253
264
275
286
297
308
319
330
341
352
363
374
385
396
407
418
429
440
451
462
473
484
495
506
517
528
539
550
561
572
583
594
605
616
627
638
649
660
671
682
693
704
715
726
737
748
759
770
781
792
803
814
825
836
847
858
869
880
891
902
913
924
935
946
957
968
979
990
1001
1012
1023
1034
1045
1056
1067
1078
1089
1100
1111
1122
1133
1144
1155
1166
1177
1188
1199
1210
1221
1232
1243
1254
1265
1276
1287
1298
1309
1320
1331
1342
1353
1364
1375
1386
1397
1408
1419
1430
1441
1452
1463
1474
1485
1496
1507
1518
1529
1540
1551
1562
1573
1584
1595
1606
1617
1628
1639
1650
1661
1672
1683
1694
1705
1716
1727
1738
1749
1760
1771
1782
1793
1804
1815
1826
1837
1848


In [37]:
new_df.to_csv('./bubbles_data.csv')